<a href="https://colab.research.google.com/github/pawan-cpu/ThreatMatrix-Analytics/blob/main/Threatmatrix_Data_Breach_%26_Cyberthreat_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In this Exploratory Analysis
## I will look into the most common methods used in these Data Breaches and find out which types of Organizations were affected the most

In [ ]:
#Threatmatrix Data Breach & Cyberthreat Analytics colab

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'list-of-top-data-breaches-2004-2021:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1569541%2F2583794%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240317%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240317T092056Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D315f7175c48346f356c7b9fc06d4a1f44ae1401eb590ba51987bfbcb1061f5c8d66a50a2f74d160078800f3287f8805a43ea0e6087954eff9bfbc48d7c229746385a0040ed0675edc4e76a23b19a787163f2b19da5d93ea5b7ac226cd1b1df459247cdd2286b3049ba34ec90b7f8a36b6e269e61203fca0d29ba78265d9c156f851b1c1615eef1de1f66cc23fa93f303924aa3e19d9c896d0e8ec3f2546b508ea0f70b311cba972e875b7895b533004b6c8f450005f8fc681389ad50231ab84f92a872f724bcee341570ee8f6e5609dbd1d434ee44343f3fe1a4c0fdb49c3c17d1e98f0758b636b238b19872b5dad6c12032b0092e351696b61a157a0bbe1f01'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 5975 bytes downloaded
Downloaded and uncompressed: list-of-top-data-breaches-2004-2021
Data source import complete.


In [ ]:
import numpy as np
import pandas as pd


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/list-of-top-data-breaches-2004-2021/DataBreaches(2004-2021).csv


In [ ]:
df = pd.read_csv('/kaggle/input/list-of-top-data-breaches-2004-2021/DataBreaches(2004-2021).csv')

In [ ]:
df

,Entity,Year,Records,Organization type,Method
0,21st Century Oncology,2016,2200000,healthcare,hacked
1,500px,2020,14870304,social networking,hacked
2,Accendo Insurance Co.,2020,175350,healthcare,poor security
3,Adobe Systems Incorporated,2013,152000000,tech,hacked
4,Adobe Inc.,2019,7500000,tech,poor security
...,...,...,...,...,...
290,Zynga,2019,173000000,social network,hacked
291,Westpac,2019,98000,financial,hacked
292,Unknown agency(believed to be tied to United S...,2020,200000000,financial,accidentally published
293,National Health Information Center (NCZI) of S...,2020,391250,healthcare,poor security


In [ ]:
df['Method'].nunique()

22

In [ ]:
df['Organization type'].nunique()

54

In [ ]:
df['Year'].nunique()

20

In [ ]:
df.dtypes

Entity               object
Year                 object
Records               int64
Organization type    object
Method               object
dtype: object

In [ ]:
df.describe()

,Records
count,2.950000e+02
mean,4.468863e+07
std,1.959801e+08
min,6.700000e+03
25%,3.075000e+05
50%,2.000000e+06
75%,2.000000e+07
max,3.000000e+09


In [ ]:
df['Method'].value_counts()

hacked                                                  156
poor security                                            36
lost / stolen media                                      32
inside job                                               18
accidentally published                                   17
lost / stolen computer                                   15
unknown                                                   4
improper setting, hacked                                  2
poor security/inside job                                  2
intentionally lost                                        1
accidentally exposed                                      1
publicly accessible Amazon Web Services (AWS) server      1
rogue contractor                                          1
misconfiguration/poor security                            1
Poor security                                             1
data exposed by misconfiguration                          1
unprotected api                         

**As we can see, hacked is the most common method used for a data breach**

In [ ]:
df['Organization type'].value_counts()

web                                                                         49
healthcare                                                                  45
financial                                                                   36
government                                                                  27
retail                                                                      20
tech                                                                        14
telecoms                                                                    12
academic                                                                    12
gaming                                                                      10
social network                                                               8
military                                                                     6
transport                                                                    5
energy                                              

**web, healthcare, financial, government and retail companies are more in numbers**

## Let's find out how web and social media companies had a data breach

In [ ]:
df[df['Organization type'] == 'web']

,Entity,Year,Records,Organization type,Method
11,Ancestry.com,2021,300000,web,poor security
15,AOL,2021,92000000,web,"inside job, hacked"
16,AOL,2021,20000000,web,accidentally published
17,AOL,2014,2400000,web,hacked
21,Ashley Madison,2015,32000000,web,hacked
24,Auction.co.kr,2008,18000000,web,hacked
33,Betfair,2010,2300000,web,hacked
41,Canva,2019,140000000,web,hacked
51,China Software Developer Network,2011,6000000,web,hacked
52,"Chinese gaming websites (three: Duowan, 7K7K, ...",2011,10000000,web,hacked


In [ ]:
df[df['Organization type'] == 'social media']

,Entity,Year,Records,Organization type,Method
233,TikTok,2020,42000000,social media,poor security
288,YouTube,2020,4000000,social media,poor security


In [ ]:
df[df['Organization type'] == 'social network']

,Entity,Year,Records,Organization type,Method
90,Facebook,2013,6000000,social network,accidentally published
91,Facebook,2018,50000000,social network,poor security
92,Facebook,2019,540000000,social network,poor security
93,Facebook,2019,1500000,social network,accidentally uploaded
94,Facebook,2019,267000000,social network,poor security
109,Google Plus,2018,500000,social network,poor security
123,Instagram,2020,200000000,social network,poor security
290,Zynga,2019,173000000,social network,hacked


**Facebook seems to have many data breaches**

In [ ]:
df[df['Entity'] == 'Facebook']

,Entity,Year,Records,Organization type,Method
90,Facebook,2013,6000000,social network,accidentally published
91,Facebook,2018,50000000,social network,poor security
92,Facebook,2019,540000000,social network,poor security
93,Facebook,2019,1500000,social network,accidentally uploaded
94,Facebook,2019,267000000,social network,poor security


In [ ]:
df['Entity'].value_counts()

Facebook                                5
AOL                                     3
Citigroup                               3
Yahoo                                   2
TD Ameritrade                           2
                                       ..
GS Caltex                               1
Hannaford Brothers Supermarket Chain    1
HauteLook                               1
Health Net                              1
Les Éditions Protégez-vous              1
Name: Entity, Length: 277, dtype: int64

It checks out that Facebook is on the top of the list followed by AOL but you can see that facebook was not hacked, it was either poor security or accidentally published data

In [ ]:
df[df['Entity'] == 'AOL']

,Entity,Year,Records,Organization type,Method
15,AOL,2021,92000000,web,"inside job, hacked"
16,AOL,2021,20000000,web,accidentally published
17,AOL,2014,2400000,web,hacked


Most AOL hacks are recent

## Let's see how many breaches happened each year

In [ ]:
df['Year'].value_counts()

2011             33
2013             26
2019             26
2020             24
2012             22
2018             21
2010             19
2014             18
2015             18
2016             17
2008             16
2009             13
2007             12
2021              9
2005              6
2017              6
2006              6
2018-2019         1
2014 and 2015     1
2004              1
Name: Year, dtype: int64

**Turns out 2011 had the highest number of breaches followed by 2019**

In [ ]:
df[df['Year'] == '2004']

,Entity,Year,Records,Organization type,Method
128,Japanet Takata,2004,510000,shopping,inside job


**This is the oldest data breach in the whole dataset**

In [ ]:
df[df['Year'] == '2011']['Method'].value_counts()

hacked                    17
lost / stolen media        5
lost / stolen computer     4
accidentally published     4
poor security              2
inside job                 1
Name: Method, dtype: int64

In [ ]:
df[df['Year'] == '2011']['Organization type'].value_counts()

healthcare              8
web                     7
gaming                  4
government              4
financial               3
retail                  2
military                2
military, healthcare    1
academic                1
media                   1
Name: Organization type, dtype: int64

**Healthcare was the second most affected industry**

In [ ]:
df[df['Organization type'] == 'healthcare']['Year'].value_counts()

2010    8
2011    8
2015    5
2009    4
2020    3
2021    3
2019    3
2016    2
2017    2
2013    2
2012    2
2007    1
2014    1
2018    1
Name: Year, dtype: int64

**Most attacks for healthcare were in 2010 and 2011**

In [ ]:
df[df['Organization type'] == 'healthcare']['Method'].value_counts()

hacked                      17
lost / stolen media         14
lost / stolen computer       6
poor security                5
inside job                   2
poor security/inside job     1
Name: Method, dtype: int64

**There were mostly hack methods used followed by stolen media or lost files**

In [ ]:
df[df['Organization type'] == 'gaming']

,Entity,Year,Records,Organization type,Method
12,Animal Jam,2020,46000000,gaming,hacked
34,Bethesda Game Studios,2011,200000,gaming,hacked
35,Blank Media Games,2018,7633234,gaming,hacked
36,Blizzard Entertainment,2012,14000000,gaming,hacked
173,Nintendo (Club Nintendo),2013,240000,gaming,hacked
174,Nintendo (Nintendo Account),2020,160000,gaming,hacked
176,Nival Networks,2016,1500000,gaming,hacked
200,Sega,2011,1290755,gaming,hacked
206,Sony Online Entertainment,2011,24600000,gaming,hacked
208,Sony PlayStation Network,2011,77000000,gaming,hacked


**Gaming industry was mostly hacked too**

In [ ]:
df[df['Organization type'] == 'gaming']['Year'].value_counts()

2011    4
2020    2
2018    1
2012    1
2013    1
2016    1
Name: Year, dtype: int64

## Financial Indutry

In [ ]:
df[df['Organization type'] == 'financial']['Year'].value_counts()

2005    5
2012    4
2008    3
2019    3
2011    3
2007    3
2010    3
2009    2
2006    2
2015    2
2014    2
2020    2
2016    1
2013    1
Name: Year, dtype: int64

In [ ]:
df[df['Organization type'] == 'financial']['Method'].value_counts()

hacked                    17
lost / stolen media        7
inside job                 6
poor security              2
unsecured S3 bucket        1
intentionally lost         1
rogue contractor           1
accidentally published     1
Name: Method, dtype: int64

## Government

In [ ]:
df[df['Organization type'] == 'government']['Year'].value_counts()

2008    6
2012    4
2011    4
2007    3
2013    3
2016    2
2010    2
2015    2
2018    1
Name: Year, dtype: int64

In [ ]:
df[df['Organization type'] == 'government']['Method'].value_counts()

hacked                    7
lost / stolen media       6
accidentally published    6
inside job                4
poor security             3
lost / stolen computer    1
Name: Method, dtype: int64

## retail

In [ ]:
df[df['Organization type'] == 'retail']['Year'].value_counts()

2007    4
2015    3
2014    3
2019    2
2011    2
2005    1
2018    1
2008    1
2016    1
2013    1
2020    1
Name: Year, dtype: int64

In [ ]:
df[df['Organization type'] == 'retail']['Method'].value_counts()

hacked                    16
lost / stolen computer     2
inside job                 1
poor security              1
Name: Method, dtype: int64

## Summary of the Analysis:
1. Most common method for used for Data Beaches is Hacking
2. Most affected companies were related to web, healthcare, financial, government, Social Media and retail companies
3. Facebook had the most data breaches, followed by Citigroup and AOL
4. None of the methods for Facebook's data breach were 'hacked', it was mostly poor security or accidentally published
5. Highest number of data breaches happened in 2011, upto 33 in total
6. 2004 only saw one data breach, lowest in the whole dataset
7. In 2010 and 2011, mostly heathcare and web industry was affected
8. Hacked was the most common method for heathcare as well but there were 7 cases where there was a stolen or lost files scenario
9. All of the gaming industry saw 'hacked' as a common method
10. Financial industry was mostly affected in 2005
11. Governments saw most breaches in 2008
12. Retail industry saw most breches in 2007